# Predicting Movie Likes and Dislikes

In [ ]:
## All important imports go here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In this project we are attempting to view different models' ability to predict likes and dislikes of movies. Below we have downloaded a dataframe from Kaggle that includes different information about movies from 1986-2016. The information includes things such as genre, company, the name of the movie, runtime, etc. What concerns us most is the score column that is included in this dataframe. The movies were each given a score that was ranked out of 10, 10 being the best, 1 being the worst. We want to utilize these scores to be able to build and train a model to predict movies scores accurately. 

In [ ]:
movies = pd.read_csv('movies.csv', encoding = 'latin-1') #Read in this data

In [ ]:
movies.head() #Display dataframe

In [ ]:
#Dropping unnecessary columns 
movies = movies.drop(columns =['country', 'rating', 'released', 'votes', 'writer', 'director', 'star'], axis =1)

In [ ]:
avg_score = movies['score'].mean()

In [ ]:
##Binarizing the movie ratings into zeros and ones for easy classification later on
movies['score'][movies['score'] < avg_score]= 0
movies['score'][movies['score'] > avg_score]= 1

In [ ]:
##Doing a linear regression to find values to replace 0.0 in the budget column 
x = movies['gross']
y = movies['budget']

x_constant = sm.add_constant(x)
gross_budget_model = sm.OLS(y, x_constant)
results = gross_budget_model.fit()
print("Intercept and slope are:", results.params)

In [ ]:
m = results.params[0]
b = results.params[1]
#Replacing budgets of 0 with the budget values calculated in linear regression model
for i in range(movies.shape[0]):
    if movies['budget'][i] ==0.0:
        gross_val = movies['gross'][i]
        y = m*gross_val + b
        movies['budget'].iloc[i] = y

In [ ]:
movies.head()

In [ ]:
#Doing OneHotEncoder for genre labels, this allows genres to be in zeros and ones so we can use them as features
#since they are no longer strings
encoder = OneHotEncoder()
genre = movies['genre']
genre_np = genre.to_numpy()
genre_ary = encoder.fit_transform(genre_np.reshape(-1,1)).toarray()

In [ ]:
genre_df = pd.DataFrame(genre_ary)

In [ ]:
genre_df = genre_df.rename({0:'Action', 1:'Adventure', 2:'Animation', 3:'Biography', 4:'Comedy', 5:'Crime', 
                            6:'Drama', 7:'Family', 8:'Fantasy', 9:'Horror', 10:'Musical', 11:'Mystery', 12:'Romance', 
                            13:'Sci-Fi', 14:'Thriller', 15:'War', 16:'Western'}, axis = 1)

In [ ]:
movies_encoded = pd.concat([movies, genre_df], axis =1)
movies_encoded.head() #OneHotEncoder worked

In [ ]:
cleaned_df = movies_encoded.drop(columns=['genre', 'company', 'name'], axis=1)
cleaned_df.head() #Cleaned dataframe ready for modeling

## PCA

In [ ]:
features = cleaned_df.drop(columns=['score'], axis =1)
labels = cleaned_df['score']

In [ ]:
train_vectors, test_vectors, train_labels, test_labels = train_test_split(features, labels, train_size =.75,
                                                                         test_size = .25, random_state =1)

We will first create a PCA model using kernel rbf along with default C and gamma values. We will also includes 10 components for this PCA which is fewer than the number of features in our dataset.

In [ ]:
##Creating and fiting PCA model, first using 10 components
pca = PCA(n_components=10, whiten=True)
pca = pca.fit(train_vectors)

##Now transforming train and test vectors into PCA train and test vectors
pca_train_vectors = pca.transform(train_vectors)
pca_test_vectors = pca.transform(test_vectors)

In [ ]:
##Now fitting model using SVC with kernel rbf with default C and gamma values
pca_svm = SVC(kernel ='rbf', C=10, gamma = 0.1)
pca_model = pca_svm.fit(pca_train_vectors, train_labels)
pca_ypred = pca_model.predict(pca_test_vectors)

In [ ]:
#Now printing metrics to look at accuracy of model
print('The confusion matrix is \n', confusion_matrix(test_labels, pca_ypred))
print('The classification report is \n', classification_report(test_labels, pca_ypred))
print('The accuracy score is \n', accuracy_score(test_labels, pca_ypred))

Now, we will attempt to make a PCA that has the same number of components as features in our dataset. We will then see if the accuracy scores change at all with this change in number of components. We will keep the kernel as rbf and default C and gamma values, same as the first PCA. 

In [ ]:
##Creating and fiting PCA model, now using 21 components, which is equal to the number of features in our dataset
pca = PCA(n_components=21, whiten=True)
pca = pca.fit(train_vectors)

##Now transforming train and test vectors into PCA train and test vectors
pca_train_vectors = pca.transform(train_vectors)
pca_test_vectors = pca.transform(test_vectors)

In [ ]:
##Now fitting model using SVC with kernel rbf with default C and gamma values
pca_svm = SVC(kernel ='rbf', C=10, gamma = 0.1)
pca_model = pca_svm.fit(pca_train_vectors, train_labels)
pca_ypred21 = pca_model.predict(pca_test_vectors)

In [ ]:
#Now printing metrics to look at accuracy of model
print('The confusion matrix is \n', confusion_matrix(test_labels, pca_ypred21))
print('The classification report is \n', classification_report(test_labels, pca_ypred21))
print('The accuracy score is \n', accuracy_score(test_labels, pca_ypred21))

In [ ]:
print('The accuracy score of the PCA with 10 components is \n', accuracy_score(test_labels, pca_ypred))
print('The accuracy score of the PCA with 21 components is \n', accuracy_score(test_labels, pca_ypred21))

As we can see in the cell above, the accuracy score for both PCAs, one with 10 components and one with 21 components, are pretty much equal. If we look more closely we can see that the accuracy for 21 components is slightly lower than the accuracy for 10 components. This might indicate that we might need to use fewer components and to get a better accuracy since after 10 components the accuracies are the same. Perhaps once we pass 10 components we might be overfitting the data, so it might be a good idea to try a PCA with fewer components, for example 4. The number of features that do not include the OneHotEncoder we did of the genres is four features, so maybe that would be a good number less than 10 that we could try.

In [ ]:
#Trying a PCA with 4 components to see if we were overfitting with 10 components
pca = PCA(n_components=4, whiten=True)
pca = pca.fit(train_vectors)

##Now transforming train and test vectors into PCA train and test vectors
pca_train_vectors = pca.transform(train_vectors)
pca_test_vectors = pca.transform(test_vectors)

In [ ]:
##Now fitting model using SVC with kernel rbf with default C and gamma values
pca_svm = SVC(kernel ='rbf', C=10, gamma = 0.1)
pca_model = pca_svm.fit(pca_train_vectors, train_labels)
pca_ypred4 = pca_model.predict(pca_test_vectors)

In [ ]:
#Now printing metrics to look at accuracy of model
print('The confusion matrix is \n', confusion_matrix(test_labels, pca_ypred4))
print('The classification report is \n', classification_report(test_labels, pca_ypred4))
print('The accuracy score is \n', accuracy_score(test_labels, pca_ypred4))

In [ ]:
print('The accuracy score for a PCA with 4 components is \n', accuracy_score(test_labels, pca_ypred4))
print('The accuracy score of the PCA with 10 components is \n', accuracy_score(test_labels, pca_ypred))
print('The accuracy score of the PCA with 21 components is \n', accuracy_score(test_labels, pca_ypred21))

This didn't work either. The accuracy with 4 components seems to be the same as the accuracy with 21 components. However, one interesting thing to look at is the confusion matrix for all three models. 

In [ ]:
print('The confusion matrix for a PCA with 4 components is \n', confusion_matrix(test_labels, pca_ypred4))
print('The confusion matrix for a PCA with 21 components is \n', confusion_matrix(test_labels, pca_ypred21))
print('The confusion matrix for a PCA with 10 components is \n', confusion_matrix(test_labels, pca_ypred))